In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Frame, PageTemplate
from reportlab.lib.units import inch
from reportlab.lib.utils import ImageReader
from PIL import Image

# Load the medicine dataset with dtype specification
medicine_dataset_path = '/content/drive/My Drive/medicine_dataset.csv'
dtype_spec = {42: str, 43: str, 44: str, 45: str, 46: str, 47: str, 48: str}  # specify appropriate columns
medicine_data = pd.read_csv(medicine_dataset_path, dtype=dtype_spec)

# Paths to images
background_image_first_page_path = '/content/Image01.jpg'  # Background image 1
background_image_other_pages_path ='/content/Image01.jpg'  # Background image 2

# Check if the image files are valid
try:
    for image_path in [background_image_first_page_path, background_image_other_pages_path]:
        with open(image_path, 'rb') as img_file:
            img = Image.open(img_file)
            img.verify()  # Verify that it is an image
except (IOError, FileNotFoundError, Image.UnidentifiedImageError) as e:
    print(f"Error with the image file: {e}")
    exit()

def generate_pdf_report(drug_name):
    drug_info = medicine_data[medicine_data['name'].str.contains(drug_name, case=False, na=False)]
    if drug_info.empty:
        print("No information found for the drug:", drug_name)
        return

    file_name = f"{drug_name}_report.pdf"
    doc = SimpleDocTemplate(file_name, pagesize=letter)

    # Styles
    styles = getSampleStyleSheet()
    normal_style = ParagraphStyle(name='Normal', fontName='Helvetica', fontSize=12, spaceAfter=12)
    header_style = ParagraphStyle(name='Header', fontName='Helvetica-Bold', fontSize=14, spaceAfter=12)

    # Story for the document content
    story = []

    # Add initial spacer to ensure the text starts from the white area
    story.append(Spacer(1, 3.5 * inch))

    # Drug Name and ID
    drug_name_heading = Paragraph(f"<b>Drug Name:</b> {drug_info['name'].values[0]}", header_style)
    drug_id = Paragraph(f"<b>ID:</b> {drug_info['id'].values[0]}", normal_style)
    story.append(drug_name_heading)
    story.append(drug_id)
    story.append(Spacer(1, 12))

    # Use
    use_heading = Paragraph("<b>Use:</b>", header_style)
    use_content = Paragraph(" ".join(drug_info[[col for col in drug_info.columns if 'use' in col]].dropna(axis=1).values[0]), normal_style)
    story.append(use_heading)
    story.append(use_content)
    story.append(Spacer(1, 12))

    # Substitutes
    substitutes_heading = Paragraph("<b>Substitutes:</b>", header_style)
    substitutes_list = drug_info[[col for col in drug_info.columns if 'substitute' in col]].dropna(axis=1).values[0]
    story.append(substitutes_heading)
    for substitute in substitutes_list:
        story.append(Paragraph(substitute, normal_style))
    story.append(Spacer(1, 12))

    # Side Effects
    side_effects_heading = Paragraph("<b>Side Effects:</b>", header_style)
    side_effects_list = drug_info[[col for col in drug_info.columns if 'sideEffect' in col]].dropna(axis=1).values[0]
    story.append(side_effects_heading)
    for side_effect in side_effects_list:
        story.append(Paragraph(side_effect, normal_style))
    story.append(Spacer(1, 12))

    # Chemical Class
    chemical_class_heading = Paragraph("<b>Chemical Class:</b>", header_style)
    chemical_class_value = drug_info['Chemical Class'].values[0] if pd.notna(drug_info['Chemical Class'].values[0]) else "Not Available"
    chemical_class_content = Paragraph(chemical_class_value, normal_style)
    story.append(chemical_class_heading)
    story.append(chemical_class_content)
    story.append(Spacer(1, 12))

    # Habit Forming
    habit_forming_heading = Paragraph("<b>Habit Forming:</b>", header_style)
    habit_forming_content = Paragraph(str(drug_info['Habit Forming'].values[0]), normal_style)
    story.append(habit_forming_heading)
    story.append(habit_forming_content)
    story.append(Spacer(1, 12))

    # Therapeutic Class
    therapeutic_class_heading = Paragraph("<b>Therapeutic Class:</b>", header_style)
    therapeutic_class_content = Paragraph(str(drug_info['Therapeutic Class'].values[0]), normal_style)
    story.append(therapeutic_class_heading)
    story.append(therapeutic_class_content)
    story.append(Spacer(1, 12))

    # Action Class
    action_class_heading = Paragraph("<b>Action Class:</b>", header_style)
    action_class_value = drug_info['Action Class'].values[0] if pd.notna(drug_info['Action Class'].values[0]) else "Not Available"
    action_class_content = Paragraph(action_class_value, normal_style)
    story.append(action_class_heading)
    story.append(action_class_content)
    story.append(Spacer(1, 12))

    # Function to add the first page background
    def add_first_page_background(canvas, doc):
        canvas.drawImage(background_image_first_page_path, 0, 0, width=letter[0], height=letter[1])

    # Function to add the other pages background
    def add_other_pages_background(canvas, doc):
        canvas.drawImage(background_image_other_pages_path, 0, 0, width=letter[0], height=letter[1])

    # Define the white space frame and move the text slightly up
    frame = Frame(inch, 3 * inch, letter[0] - 2 * inch, letter[1] - 3.5 * inch, id='white_space_frame')

    # Define the page templates with background and frame
    first_page_template = PageTemplate(id='FirstPage', frames=[frame], onPage=add_first_page_background)
    other_pages_template = PageTemplate(id='OtherPages', frames=[frame], onPage=add_other_pages_background)
    doc.addPageTemplates([first_page_template, other_pages_template])

    # Build the PDF
    doc.build(story)

    print(f"PDF report generated: {file_name}")

drug_name = input("Enter the drug name: ")
generate_pdf_report(drug_name)

Enter the drug name: duac
PDF report generated: duac_report.pdf
